In [1]:
import json
import requests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from math import ceil

In [2]:
# Токены для доступа к api
token1 = '***'
token2 = '***'
url = "http://api.duma.gov.ru/api/{token1}/deputies.json?app_token={token2}&position=Депутат%20ГД".format(token1=token1, token2=token2)

In [3]:
# Получение списка депутатов
req = requests.get(url)
deputies = json_normalize(json.loads(req.text), ['factions'], ['id', 'name', 'position'], record_prefix='_')
deputies['_endDate'] = pd.to_datetime(deputies['_endDate'])
deputies['_startDate'] = pd.to_datetime(deputies['_startDate'])

In [4]:
deputies

,_id,_name,_startDate,_endDate,id,name,position
0,72100004,"Фракция Политической партии ""КОММУНИСТИЧЕСКАЯ ...",2011-12-04,2016-10-04,99111772,Абалаков Александр Николаевич,Депутат ГД
1,72100024,"Фракция Всероссийской политической партии ""ЕДИ...",2013-03-06,2016-10-04,99112284,Абасов Мамед Магарамович,Депутат ГД
2,72100020,"Депутатская группа ""Российские регионы""",1996-01-16,1997-11-12,99100491,Абдулатипов Рамазан Гаджимурадович,Депутат ГД
3,72100024,"Фракция Всероссийской политической партии ""ЕДИ...",2011-12-04,2013-01-27,99100491,Абдулатипов Рамазан Гаджимурадович,Депутат ГД
4,72100001,Фракция Аграрной партии России,1994-01-12,1994-07-23,99100001,Абдулбасиров Магомедтагир Меджидович,Депутат ГД
...,...,...,...,...,...,...,...
4776,72100024,"Фракция Всероссийской политической партии ""ЕДИ...",2007-12-24,2011-12-20,99111094,Яхихажиев Саид Кожалович,Депутат ГД
4777,72100024,"Фракция Всероссийской политической партии ""ЕДИ...",2017-09-10,2021-12-20,99113127,Яхнюк Сергей Васильевич,Депутат ГД
4778,72100009,"Фракция ""Единство - Единая Россия""",2000-01-18,2003-12-28,99100900,Яшин Александр Михайлович,Депутат ГД
4779,72100005,Фракция Политической партии ЛДПР - Либерально-...,1996-12-05,2000-01-17,99107185,Яшин Виктор Анатольевич,Депутат ГД


In [5]:
deputies_id = set(deputies['id'])
len(deputies_id)

2023

In [6]:
%%time
# Получение данных из api
url2 = "http://api.duma.gov.ru/api/c2260578338f436d0791f702d65bfc4e0fa021b1/transcriptDeputy/{}.json?app_token=appc0b7ceeb2e3810948b192460ee6d4fe6f348c0d3&page={}&limit=20&dateFrom=1990-01-01&dateTo=2021-03-20"
total = []
totalizer = 0
for s in deputies_id:
    r2 = requests.get(url2.format(s,1))
    tot = json.loads(r2.text)
    tot['deputies'] = s
    total.append(tot)
    page = ceil(int(json.loads(r2.text)['totalCount']) / 20) if json.loads(r2.text)['totalCount'] is not None else 1
    if page > 1:
        for i in list(range(2,page+1)):
            r2 = requests.get(url2.format(s,i))
            tot = json.loads(r2.text)
            tot['deputies'] = s
            total.append(tot)
            totalizer += 1
    totalizer += 1

Wall time: 7h 54min 2s


In [7]:
# Количество итераций при прохождении api
totalizer

19121

In [8]:
data = json_normalize(total,['meetings', 'questions', 'parts' ],  ['name', 'pageSize', 'page', 'totalCount', 'questionCount', ['meetings','date'], 'deputies'], record_prefix='_')

In [9]:
data

,_startLine,_endLine,_type,_lines,_votes,name,pageSize,page,totalCount,questionCount,meetings.date,deputies
0,1043,1051,,"[КОЛЕСНИКОВ В. И., фракция ""ЕДИНАЯ РОССИЯ"". ...",[],Колесников Виктор Иванович,20,1,348,193,2011-11-18 00:00:00,99102994
1,1203,1212,65100003,[КОЛЕСНИКОВ В. И. Спасибо. ...,[],Колесников Виктор Иванович,20,1,348,193,2011-11-18 00:00:00,99102994
2,4581,4593,65100003,[КОЛЕСНИКОВ В. И. Уважаемый председательствующ...,[],Колесников Виктор Иванович,20,1,348,193,2011-11-18 00:00:00,99102994
3,1819,1830,65100003,"[КОЛЕСНИКОВ В. И., фракция ""ЕДИНАЯ РОССИЯ"". ...",[],Колесников Виктор Иванович,20,1,348,193,2011-10-21 00:00:00,99102994
4,2591,2617,65100003,"[КОЛЕСНИКОВ В. И., фракция ""ЕДИНАЯ РОССИЯ"". ...",[],Колесников Виктор Иванович,20,1,348,193,2011-10-18 00:00:00,99102994
...,...,...,...,...,...,...,...,...,...,...,...,...
359325,3535,3536,,[КУВАЕВ А. А. Нет. ...,[],Куваев Александр Александрович,20,2,25,20,2000-09-20 00:00:00,99103327
359326,882,885,,"[КУВАЕВ А. А., фракция Коммунистической партии...",[],Куваев Александр Александрович,20,2,25,20,2000-07-07 00:00:00,99103327
359327,450,466,,"[КУВАЕВ А.А., фракция Коммунистической партии ...",[],Куваев Александр Александрович,20,2,25,20,1998-09-09 00:00:00,99103327
359328,667,673,,"[КУВАЕВ А.А., фракция Коммунистической партии ...",[],Куваев Александр Александрович,20,2,25,20,1997-09-03 00:00:00,99103327


In [10]:
%%time
# Добавление столбца с названиями партий
def merg(x):
    try:
        return deputies[(deputies['id'] == x['deputies']) & (deputies['_startDate'] <= x['meetings.date']) & (deputies['_endDate'] >= x['meetings.date'])]['_name'].values[0]
    except:
        return None
data['party'] = data.apply(merg, axis=1)

Wall time: 23min 12s


In [11]:
data

,_startLine,_endLine,_type,_lines,_votes,name,pageSize,page,totalCount,questionCount,meetings.date,deputies,party
0,1043,1051,,"[КОЛЕСНИКОВ В. И., фракция ""ЕДИНАЯ РОССИЯ"". ...",[],Колесников Виктор Иванович,20,1,348,193,2011-11-18 00:00:00,99102994,None
1,1203,1212,65100003,[КОЛЕСНИКОВ В. И. Спасибо. ...,[],Колесников Виктор Иванович,20,1,348,193,2011-11-18 00:00:00,99102994,None
2,4581,4593,65100003,[КОЛЕСНИКОВ В. И. Уважаемый председательствующ...,[],Колесников Виктор Иванович,20,1,348,193,2011-11-18 00:00:00,99102994,None
3,1819,1830,65100003,"[КОЛЕСНИКОВ В. И., фракция ""ЕДИНАЯ РОССИЯ"". ...",[],Колесников Виктор Иванович,20,1,348,193,2011-10-21 00:00:00,99102994,None
4,2591,2617,65100003,"[КОЛЕСНИКОВ В. И., фракция ""ЕДИНАЯ РОССИЯ"". ...",[],Колесников Виктор Иванович,20,1,348,193,2011-10-18 00:00:00,99102994,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
359325,3535,3536,,[КУВАЕВ А. А. Нет. ...,[],Куваев Александр Александрович,20,2,25,20,2000-09-20 00:00:00,99103327,"Фракция Политической партии ""КОММУНИСТИЧЕСКАЯ ..."
359326,882,885,,"[КУВАЕВ А. А., фракция Коммунистической партии...",[],Куваев Александр Александрович,20,2,25,20,2000-07-07 00:00:00,99103327,"Фракция Политической партии ""КОММУНИСТИЧЕСКАЯ ..."
359327,450,466,,"[КУВАЕВ А.А., фракция Коммунистической партии ...",[],Куваев Александр Александрович,20,2,25,20,1998-09-09 00:00:00,99103327,"Фракция Политической партии ""КОММУНИСТИЧЕСКАЯ ..."
359328,667,673,,"[КУВАЕВ А.А., фракция Коммунистической партии ...",[],Куваев Александр Александрович,20,2,25,20,1997-09-03 00:00:00,99103327,"Фракция Политической партии ""КОММУНИСТИЧЕСКАЯ ..."


In [12]:
# Сохранение json и датафрейма
import pickle
with open('duma/data_dataframe.pickle', 'wb') as dep:
    pickle.dump(data, dep)
    
with open('duma/total_json.pickle', 'wb') as dep:
    pickle.dump(total, dep)